<a href="https://colab.research.google.com/github/HarryKao1020/stock_trading_strategy/blob/colab/backtesting/weekly_five.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib


!pip install backtesting
!pip install shioaji
!pip install python-dotenv
import numpy as np
import pandas as pd
import requests
import shioaji as sj
import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
# import talib
import os
from dotenv import load_dotenv

from google.colab import userdata

# 加載 .env 文件中的環境變數
load_dotenv()

# 建立API物件，simulation=True是代表測試帳號
api = sj.Shioaji(simulation=True)

#==== for jupyter
# shioaji_secret = os.getenv("SHIOAJI_SECRETKEY", None)
# shioaji_apikey = os.getenv("SHIOAJI_APIKEY", None)

#==== for Colab
shioaji_secret=userdata.get('SHIOAJI_SECRETKEY')
shioaji_apikey=userdata.get('SHIOAJI_APIKEY')



# 登入你的key
# accounts = api.login("YOUR_API_KEY", "YOUR_SECRET_KEY")
accounts = api.login(shioaji_apikey, shioaji_secret)

--2024-07-26 03:31:38--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-07-26 03:31:38--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-07-26 03:31:38--  http://netactuate.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1
Res

2024-07-26 03:35:43.708 | WARNING  | importlib._bootstrap:_call_with_frames_removed:241 - Optional: pip install shioaji[speed] for better performance.
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [2]:
#====定義日期 ex:1年,2年,3年======
today = date.today()
startDate = date(2024,3,4)
oneYear= 365
twoYear = 730
delta_oneYear = timedelta(days=oneYear)
delta_twoYear = timedelta(days=twoYear)
date_oneYear_ago = today - delta_oneYear
date_twoYear_age = today - delta_twoYear
testDate=startDate-timedelta(days=3)
lastWeek=startDate-timedelta(days=10)

# ==== end
# ====篩選週漲幅20% & 成值前100名 ====
##成值前100
scanners = api.scanners(
    scanner_type=sj.constant.ScannerType.AmountRank,
    count=100,
    date=str(startDate)
)
scanners
amout_df = pd.DataFrame(s.__dict__ for s in scanners)
amout_df.ts = pd.to_datetime(amout_df.ts)
stock_code_array = amout_df['code'].values


##成值前100 & 週漲幅20%(含以上）
amount_raise_stock = []
def raise_twenty_percent(stock_code):
    kbars = api.kbars(
        contract=api.Contracts.Stocks[stock_code],
        start=str(lastWeek),
        end=str(testDate),
    )
    df = pd.DataFrame({**kbars})
    df.ts = pd.to_datetime(df.ts)
    df.set_index('ts',inplace=True)

    # 取週用resample('W') ,週k最後一天13:30的K棒 , dropna清除NaN的值
    weekly_df=df.resample('W').last().dropna()
    last_week_close = weekly_df['Close'].iloc[-2]
    this_week_close = weekly_df['Close'].iloc[-1]
    if this_week_close >= last_week_close*1.2:
        amount_raise_stock.append(stock_code)


for stock_code in stock_code_array:
    raise_twenty_percent(stock_code)
print(amount_raise_stock)
# ====end


['3324', '6806', '3147', '7402']


In [3]:
# 取得週k線
def weekly_average(stock_code):
  kbars = api.kbars(
      contract=api.Contracts.Stocks[stock_code],
      start=str(startDate),
      end=str(today),
  )
  df = pd.DataFrame({**kbars})
  df.ts = pd.to_datetime(df.ts)
  df.set_index('ts',inplace=True)


  # 取週用resample('W') ,週k最後一天13:30的K棒 , dropna清除NaN的值
  weekly_df=df.resample('W').last().dropna()
  print(weekly_df)
  return weekly_df

def daily_average(stock_code):
  kbars = api.kbars(
      contract=api.Contracts.Stocks[stock_code],
       start=str(startDate),
      end=str(today),
  )
  df = pd.DataFrame({**kbars})
  df.ts = pd.to_datetime(df.ts)
  df.set_index('ts',inplace=True)


  # 取週用resample('W') ,週k最後一天13:30的K棒 , dropna清除NaN的值
  daily_df=df.resample('D').last().dropna()
  return daily_df



# ==== 5ma strategy =====
class OneMA(Strategy):
  # 定義均線參數
  n1=5
  n2=10
  def init(self):
    self.sma5 = self.I(SMA, self.data.Close,self.n1)
    self.sma10 = self.I(SMA, self.data.Close, self.n2)

  def next(self):
    if(self.data.Close > self.sma5) and (not self.position.is_long):
      self.buy()
    elif (self.data.Close <self.sma5):
      self.position.close()

backtest_df = weekly_average("6806")

bt = Backtest(backtest_df,OneMA, cash=300000, commission=0.002)

# #將跑完回測得到的數據放到stats
stats = bt.run()
print(stats)
bt.plot(superimpose = False)


             Open   High    Low  Close  Volume       Amount
ts                                                         
2024-03-10  146.5  146.5  146.5  146.5     585   85702500.0
2024-03-17  138.0  138.0  138.0  138.0     504   69552000.0
2024-03-24  147.0  147.0  147.0  147.0     554   81438000.0
2024-03-31  144.0  144.0  144.0  144.0     224   32256000.0
2024-04-07  151.0  151.0  151.0  151.0     183   27633000.0
2024-04-14  150.0  150.0  150.0  150.0     367   55050000.0
2024-04-21  146.5  146.5  146.5  146.5     293   42924500.0
2024-04-28  138.0  138.0  138.0  138.0     292   40296000.0
2024-05-05  131.5  131.5  131.5  131.5      80   10520000.0
2024-05-12  126.5  126.5  126.5  126.5     107   13535500.0
2024-05-19  121.0  121.0  121.0  121.0     109   13189000.0
2024-05-26  125.5  125.5  125.5  125.5      81   10165500.0
2024-06-02  126.0  126.0  126.0  126.0      79    9954000.0
2024-06-09  131.5  131.5  131.5  131.5     201   26431500.0
2024-06-16  137.5  137.5  137.5  137.5  

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1338', ...)

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1925', ...)